## Resumo Descritivo do Código

O código realiza o processamento dos dados de setores de coleta obtidos por meio de webscraping. Ele lê arquivos no formato **JSON Lines (JSONL)** contendo informações sobre setores, horários de coleta, coordenadas de referência e polígonos geográficos que representam a área atendida. A partir dessa leitura inicial, o código organiza, transforma e estrutura esses dados em um **GeoDataFrame** padronizado para posterior análise espacial e armazenamento em formato geoespacial.

---

Após a leitura do  arquivo `setores_final.jsonl`,convertendo-o em um DataFrame estruturado. Em seguida, remove-se o prefixo "Setor " dos nomes das áreas de coleta para manter apenas o código numérico do setor.

O processamento dos dados ocorre em três etapas principais:

**1. TRATAMENTO DOS HORÁRIOS E DIAS DE COLETA**
* O período de coleta (diurno ou noturno), inferido com base na hora inicial;
* A lista completa de dias da semana em que há coleta no setor.
Os resultados são integrados ao DataFrame original nas colunas horario e dias_da_semana.

**2.  EXTRAÇÃO E ORGANIZAÇÃO DOS POLÍGONOS GEOGRÁFICOS**

Toda a lógica referente à obtenção das geometrias dos setores é consolidada em uma única etapa, englobando tanto a extração quanto a normalização dos registros.

* **Extração de todos os polígonos em `area_json`**\
  Uma função percorre cada registro e identifica as geometrias (**Polygon** e **MultiPolygon** ). Cada geometria encontrada é convertida em um objeto `shapely.geometry.Polygon`.

* **Normalização dos registros (uma linha por polígono)**\
  Alguns setores contêm múltiplos polígonos. Assim, o código cria uma  **uma linha independente para cada polígono extraído**. Ao final do processo,cada linha resultante inclui:

    - Código da área de coleta (`seq_area_coleta`);
    - Número do setor (`nom_area_coleta`);
    - Período da coleta (`horario`);
    - Dias da semana (`dias_da_semana`);
    - Coordenadas de referência (`position.latitude`, `position.longitude`);
    - A geometria correspondente (`Polygon`).

Os registros são reunidos em um **GeoDataFrame** com sistema de referência espacial **EPSG:4326** (WGS84).

**3. EXPORTAÇÃO FINAL**\
O GeoDataFrame consolidado é exportado em formato **GeoPackage (GPKG)** com o nome `setores_coleta.gpkg`.  
O script verifica previamente se o arquivo já existe, evitando sobrescrita acidental.


In [1]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import folium
import os

In [2]:
path='/home/akel/PycharmProjects/Data_ambiental/data/raw'
arquivo = os.path.join(path, 'setores_final.jsonl')
df = pd.read_json(arquivo, orient="records", lines=True)
df.head(5)

,seq_area_coleta,nom_area_coleta,area_json,area_coleta_horario,position.latitude,position.longitude
0,231,Setor 1406,"[{'type': 'Feature', 'geometry': {'type': 'Pol...","[{'num_dia_semana': 1, 'num_hora_inicial': '18...",-1.457621,-48.488588
1,287,Setor 1305,"[{'type': 'Feature', 'geometry': {'type': 'Pol...","[{'num_dia_semana': 1, 'num_hora_inicial': '06...",-1.475703,-48.474152
2,215,Setor 1307,"[{'type': 'Feature', 'geometry': {'type': 'Pol...","[{'num_dia_semana': 1, 'num_hora_inicial': '06...",-1.475703,-48.465140
3,242,Setor 1301,"[{'type': 'Feature', 'geometry': {'type': 'Pol...","[{'num_dia_semana': 1, 'num_hora_inicial': '06...",-1.474577,-48.497808
4,253,Setor 1302,"[{'type': 'Feature', 'geometry': {'type': 'Pol...","[{'num_dia_semana': 1, 'num_hora_inicial': '06...",-1.472325,-48.486543


In [3]:
#removendo o prefixo 'Setor ' da feature nom_area_coleta
df['nom_area_coleta'] = df['nom_area_coleta'].str.replace('Setor ', '', regex=False).str.strip()

In [4]:
#funções
# -----------------------------------------------------
# 1) PROCESSAR HORÁRIOS E DIAS
# -----------------------------------------------------
def processar_horarios_dataframe(df):
    """
    Processa area_coleta_horario em duas listas: horarios (diurno/noturno)
    e dias_da_semana (lista de dias).
    """
    horarios_resultado = []
    dias_resultado = []
    
    for horarios_list in df['area_coleta_horario']:
        if not horarios_list:
            horarios_resultado.append(None)
            dias_resultado.append(None)
        else:
            # Primeiro horário define o período
            horario = horarios_list[0]['num_hora_inicial'] 
            horario = 'diurno' if horario == '06:00:00' else 'noturno'

            # Lista de dias da semana
            dias_semana = [h['num_dia_semana'] for h in horarios_list]

            horarios_resultado.append(horario)
            dias_resultado.append(dias_semana)
    
    return horarios_resultado, dias_resultado


# Aplicar no DF
horarios, dias = processar_horarios_dataframe(df)
df['horario'] = horarios
df['dias_da_semana'] = dias


# -----------------------------------------------------
# 2) EXTRAIR *TODOS* OS POLÍGONOS DE UM REGISTRO
# -----------------------------------------------------
def extrair_poligonos(area_json):
    """
    Extrai TODOS os polígonos presentes em area_json, mesmo se houver mais de um.
    Retorna uma lista de objetos shapely.geometry.Polygon.
    """
    polys = []

    if not area_json:
        return polys

    for feature in area_json:
        geom_data = feature.get("geometry", {})
        tipo = geom_data.get("type")

        # Polygon simples
        if tipo == "Polygon":
            coords = geom_data["coordinates"][0]
            polys.append(Polygon(coords))

        # MultiPolygon (se aparecer no futuro)
        elif tipo == "MultiPolygon":
            for part in geom_data["coordinates"]:
                polys.append(Polygon(part[0]))

    return polys


# -----------------------------------------------------
# 3) CRIAR UMA LINHA POR POLÍGONO
# -----------------------------------------------------
linhas = []

for idx, row in df.iterrows():
    poligonos = extrair_poligonos(row['area_json'])

    for poly in poligonos:
        linhas.append({
            "area_coleta": row['seq_area_coleta'],
            "setor": row['nom_area_coleta'],
            "horario": row['horario'],
            "dias": row['dias_da_semana'],
            "latitude_ref": row['position.latitude'],
            "longitude_ref": row['position.longitude'],
            "geometry": poly
        })

# GeoDataFrame final
gdf = gpd.GeoDataFrame(linhas, crs="EPSG:4326")
gdf.head()

,area_coleta,setor,horario,dias,latitude_ref,longitude_ref,geometry
0,231,1406,noturno,"[1, 2, 3, 4, 5, 6]",-1.457621,-48.488588,"POLYGON ((-48.48221 -1.45644, -48.48225 -1.456..."
1,287,1305,diurno,"[1, 2, 3, 4, 5, 6]",-1.475703,-48.474152,"POLYGON ((-48.47705 -1.47251, -48.47623 -1.471..."
2,215,1307,diurno,"[1, 2, 3, 4, 5, 6]",-1.475703,-48.465140,"POLYGON ((-48.46102 -1.47104, -48.46169 -1.471..."
3,242,1301,diurno,"[1, 2, 3, 4, 5, 6]",-1.474577,-48.497808,"POLYGON ((-48.50046 -1.46973, -48.49938 -1.467..."
4,253,1302,diurno,"[1, 2, 3, 4, 5, 6]",-1.472325,-48.486543,"POLYGON ((-48.48374 -1.46823, -48.48416 -1.469..."


In [5]:
# -----------------------------------------------------
# 4)SALVAR O ARQUIVO
# -----------------------------------------------------
filename="shape_coleta.gpkg"
path='/home/akel/PycharmProjects/Data_ambiental/data/process/'
arquivo = os.path.join(path, filename)

if os.path.exists(arquivo):
    print("GPKG já existe. Não sobrescrever.")
else:
    gdf.to_file(arquivo, driver="GPKG")
    print("Arquivo GPKG salvo!")

Arquivo GPKG salvo!
